In [1]:
import os
from tqdm import tqdm

from pathlib import Path

os.chdir("..")

In [2]:
from open_extract.llm import Extractor
from open_extract.data_model import Screening

In [13]:
md_files = list(Path("data/screening_250205/mds").glob("*.md"))
print(f"{len(md_files)=}")
extractor = Extractor(model_name="long-context-deepseek", ollama_host="olvi-1:11434", target_model=Screening)

In [ ]:
# Check max words

max_words = 0
for f in md_files:
    text = f.read_text()
    words = len(text.split())
    if words > max_words:
        max_words = words

print(f"{max_words=}")
# 64k max... need to do RAG... screening should be ok...


In [ ]:
for file in tqdm(md_files):
    y = extractor.extract(file.read_text())
    file.with_suffix(".json").write_text(y.model_dump_json())